## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

## Chapter-011-23-Control-03

In [ ]:
#!pip install harold

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline

In [ ]:
import numpy as np
from harold import *
import matplotlib.pyplot as plt

# Creating models

harold defines two classes to represent the dynamic models: Transfer and State classes. 

In [ ]:
from IPython.display import Image
Image(filename='../../data/chapter 06/TransferState.png') 

In [ ]:
G1 = Transfer([1, -1],[1, -2, 1, 0], dt=0.1) # discrete
G2 = Transfer([[1, [1, 3]],[0, [1, 2]]], [[[1, 2], [1, 0, -4]],[1, [1, -3]]])
G3 = State([[0, 1], [-0.1, -0.5]], [[0], [1]], [0, 3.5], 1, dt=0.1) # discrete
G4 = State([[-4, -2, 2], [0, -5, 2], [4, -3, -4]], [[1, 1], [0, 1], [2, -1]], [-1, 5, 2])

You convert the list objects into NumPy arrays. 
Note that you are skipping the  matrix on Discrete-time models since it is zero. 

In [ ]:
print(G1)

You can directly identify the pole zero structure with the process.
The MIMO state representations is preferred exclusively, MIMO transfer models explicitly require a list of lists structure. 

In [ ]:
from IPython.display import Image
Image(filename='../../data/chapter 06/TransferStateG5.png') 

In [ ]:
G5 = Transfer([[[1, 1], 2], [[1, 3], 4]], [1, 5, 1])
print('Numerator :\n', G5.num, '\nDenominator :\n', G5.den)

You should now slice the array based on the shape of the C array and arrive at the same results

In [ ]:
M = np.array([[-4, -2, 2, 1, 1], [0, -5, 2, 0, 1], [4, -3, -4, 2, -1], [-1, 5, 2, 0, 0]])
G4 = State(*matrix_slice(M, corner_shape=[3, 3], corner='nw'))
print(G4.matrices)

In [ ]:
M = np.array([[-4, -2, 2, 1, 1],
              [0, -5, 2, 0, 1],
              [4, -3, -4, 2, -1],
              [-1, 5, 2, 0, 0]])
G4 = State(*matrix_slice(M, corner_shape=[1, 3], corner='sw'))

Further properties can now beeen accessed via the common dot syntax.
You should now investigate the model data of G1

In [ ]:
print(G1.polynomials)

You should investigate the model data of a State representation, for G3 next or simply look at the  matrix of G4

In [ ]:
print(G3.matrices)

In [ ]:
print(G4.a)

# Random State models

Finally you can create random State models via the random_stable_model

You should force the random model to have more chance to have oscillatory modes by changing the probability distribution of the selected poles. 
Assume that you want a discrete-time model with majority poles on the imaginary axis and occasional integrators.

In [ ]:
G = random_state_model(5, p=3, m=2)
print(G)

In [ ]:
G = random_state_model(20, p=3, m=2, dt=0.01, prob_dist=[0, 0, 0.1, 0.9],stable=False) # 90% osc. modes, 10% integrators
print(G)

# Conversions of Models

In order to convert one model representation to another, the relevant functions are transfer_to_state() and state_to_transfer(). 

Note: Typically the conversion from a Transfer model to a State model leads to a non-minimal representations.

In [ ]:
H = transfer_to_state(G2)
print(H)

In [ ]:
J = state_to_transfer(G3)
print(J)

# Minimal Realization

The minimality is an essential property of any representations for reliable computations and hence we can use minimal_realization() function. This function uses a distance metric to the closest rank-deficient matrix pencil for State models and a straightforward walk-over the poles and zeros for Transfer representations. 

Note: The tolerance for decision can be adjusted.

Note: MIMO poles and zeros don't necessarily cancel each other even though the values are identical. This is because for MIMO representations pole and zero directionalities can be different. You can see this for the representation H, that you obtained before.

In [ ]:
Hm = minimal_realization(H)
print(Hm)

# Discretization and undiscretization

Discretization of models is mostly an art or result of experience than pure science. Though the methods are sound, selecting the right method and the relevant sampling period is up to machine learners's choice.

In [ ]:
G6 = Transfer([1], [1, 1.4, 1])
G6d = discretize(G6, dt=0.3, method='zoh')
print(G6d)

Here is the current known discretization methods:

In [ ]:
from harold._global_constants import _KnownDiscretizationMethods as km
print('Discretization methods \n=============================')
print(*km, sep='\n')

You can use these against G3. You did not provide any method but for G6d the default method was implemented.

In [ ]:
print(G3.DiscretizedWith is None)

In [ ]:
print(G6d.DiscretizedWith)

You can now convert back these models, G3 will be converted using the default method tustin however G6d will be converted via zero-order hold method. 

Note: Had this information not present, the resulting continuous model would be slightly different than what you started with.

In [ ]:
G6dcz = undiscretize(G6d)
print(G6dcz)

In [ ]:
G6dct = undiscretize(G6d, method='tustin')
print(G6dct)

# Model algebra

Basic algebraic operations and feedback operations are implemented via typical *,+,-,@ and feedback() function. The shape and sampling time compatibility is checked before the operations and errors are raised in case of mismatches.
One exception is the operation with a SISO model over a MIMO model. For example, G1 is a SISO model and G3 is a sampling time matching MIMO model, hence following is allowed which multiplies each entry of G3 with G1. For multiplication of MIMO models matrix multiplication rules and hence matrix multiplication operator @ is followed.

In [ ]:
print(G1 * G3)

In [ ]:
print(G4 @ G2)

In [ ]:
CL = feedback(G3, G1)
CL_min = minimal_realization(CL)
print(CL)

You should observe that there is a pole/zero cancellation at 1. Which is removed afterwards in the process.

In [ ]:
print(CL_min)

# Basic Plotting Functionality

The common plots that are needed are already available.

For frequency domain plotting, the default units are Hz and powers of ten for amplitudes.

In [ ]:
impulse_response_plot(G4)
plt.show()

The discrete-time plant plots are automatically drawn as stairs.

In [ ]:
G4_d = discretize(G4, 0.1, method='zoh')
impulse_response_plot(G4_d)
plt.show()

The plot units can be changed via the keywords available

In [ ]:
bode_plot(G4)
plt.show()

In [ ]:
bode_plot(G4, use_db=True, use_hz=False, use_degree=False)
plt.show()

In [ ]:
nyquist_plot(G2)
plt.show()

# An LQR example

Just to demonstrate to you what you could perform, i will guide you through a LQR example for Inverted Pendulum: State-Space Methods for Controller Design.

In [ ]:
# Define some parameters
M, m, b, I, g, l = 0.5, 0.2, 0.1, 0.006, 9.8, 0.3
p = I*(M+m) + M*m*l**2  #denominator for the A and B matrices

A = np.array([[0, 1, 0, 0], [0, -(I+m*l**2)*b/p, (m**2*g*l**2)/p, 0],[0, 0, 0, 1], [0, -(m*l*b)/p, m*g*l*(M+m)/p, 0]])
B = np.array([[0], [(I+m*l**2)/p], [0], [m*l/p]])
C = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])

sys_ss = State(A,B,C)
print('The system is Kalman controllable:', is_kalman_controllable(sys_ss))

Q = C.T @ C
K, X, eigs = lqr(sys_ss, Q)  # R = 1 if omitted
print('Controller K gains : ', K)
sys_cl = State(A-B@K, B, C)

t = np.arange(0, 5, 0.01)
r =0.2*np.ones(len(t))
y, t = simulate_linear_system(sys_cl, u=r, t=t)
fig, ax = plt.subplots(1, 1)
ax.plot(t,y);
ax.grid(which='both')
plt.show()

Now, you increase the weights on the position states

In [ ]:
Q[[0, 2], [0, 2]] = [5000, 100]
K, X, eigs = lqr(sys_ss, Q)
print('New Controller gains : ', K)
sys_cl = State(A-B@K, B, C)
y, t = simulate_linear_system(sys_cl, u=r, t=t)
fig, ax = plt.subplots(1, 1)
ax.plot(t,y);
ax.grid(which='both')
plt.show()

## Done

In [ ]:
import datetime
now = datetime.datetime.now()
print('Done!',str(now))